# yak

> an opinonated, but configurable, data model for building JupyterLite sites

In [ ]:
import graphlib
import json
import os
import pprint
import re
import sys
from copy import deepcopy
from pathlib import Path
from typing import Any, Callable

import importnb
import traitlets as T
import yaml
from traitlets.utils.nested_update import nested_update

import jupyak

try:
    import tomllib
except:
    import tomli as tomllib

with importnb.Notebook():
    from jupyak.tasks import _well_known as W

## types

In [ ]:
TRepoFactory = Callable[["Yak"], dict]

## traits

> lifted from `ipywidgets.trait_types`

In [ ]:
class InstanceDict(T.Instance):

    """An instance trait which coerces a dict to an instance.

    This lets the instance be specified as a dict, which is used
    to initialize the instance.

    Also, we default to a trivial instance, even if args and kwargs
    is not specified.
    """

    def validate(self, obj, value):
        if isinstance(value, dict):
            return super().validate(obj, self.klass(**value))
        else:
            return super().validate(obj, value)

    def make_dynamic_default(self):
        return self.klass(*(self.default_args or ()), **(self.default_kwargs or {}))


class TypedTuple(T.Container):

    """A trait for a tuple of any length with type-checked elements."""

    klass = tuple
    _cast_types = (list,)

In [ ]:
class UnicodeWithRegex(T.Unicode):

    """A regex-constrained string"""

    pattern: re.Pattern

    def __init__(
        self,
        default_value: "t.Any" = None,
        allow_none: "bool" = False,
        read_only: "bool | None" = None,
        help: "str | None" = None,
        config: "t.Any" = None,
        **kwargs: Any,
    ) -> None:
        pattern = kwargs.pop("pattern")
        super().__init__(
            default_value=default_value,
            allow_none=allow_none,
            read_only=read_only,
            help=help,
            config=config,
            **kwargs,
        )
        self.pattern = pattern

    @property
    def info_text(self):
        return f"a string matching {self.pattern}"

    def validate(self, obj: Any, value: Any) -> str | None:
        if re.findall(self.pattern, value):
            return value
        self.error(obj, value)

Special dollar-sign prefixed values. 

In [ ]:
class SCHEMA:
    SCHEMA = "$schema"
    DEFS = "$defs"
    REF = "$ref"
    ID = "$id"
    DESC = "description"
    TITLE = "title"
    DEFAULT = "default"
    PROPS = "properties"
    ADD_PROPS = "additionalProperties"
    ITEMS = "items"
    ADD_ITEMS = "additionalItems"
    PATT_PROPS = "patternProperties"
    ALL_OF = "allOf"
    ONE_OF = "oneOf"
    ANY_OF = "anyOf"
    NULL = "null"
    CONST = "const"

JSON Schema drafts supported by `HasSchema`.

In [ ]:
class Draft7(SCHEMA):
    DRAFT = "http://json-schema.org/draft-07/schema#"

A JSON Schema-aware base class for `HasTraits`.

In [ ]:
class HasSchema(T.HasTraits):
    S = Draft7

    @classmethod
    def to_schema(cls, defs: dict | None = None, instance=None):
        """Make an ``object`` schema for this class with its traits as ``properties``."""
        root = defs is None
        defs = {} if root else defs
        ref = f"#/{cls.S.DEFS}/{cls.__name__}"
        if cls.__name__ in defs:
            return {cls.S.REF: ref}
        schema = {cls.S.TITLE: cls.__name__, cls.S.PROPS: {}}
        if not root:
            defs[cls.__name__] = schema
        if cls.__doc__:
            schema[cls.S.DESC] = cls.__doc__.strip()
        for trait_name, trait_def in cls._traits.items():
            if trait_def.metadata.get("schema") == False:
                continue
            schema[cls.S.PROPS][trait_name] = cls.trait_to_schema(
                trait_def, defs, trait_name=trait_name
            )
        cls.finalize_schema(schema, defs, instance)
        if root:
            schema.update({cls.S.SCHEMA: cls.S.DRAFT, cls.S.DEFS: defs})
        return schema

    @classmethod
    def finalize_schema(cls, schema, defs, instance):
        return schema

    @classmethod
    def dict_like_to_schema(cls, schema, trait_def, defs):
        if trait_def.klass in HasSchema.__subclasses__():
            def_name = trait_def.klass.__name__
            if def_name not in defs:
                defs[def_name] = trait_def.klass.to_schema(defs)
            schema[cls.S.REF] = f"#/{cls.S.DEFS}/{def_name}"

    @classmethod
    def trait_to_schema(cls, trait_def, defs, trait_name: str | None = None):
        schema = {}
        if trait_name:
            schema[cls.S.TITLE] = trait_name
        help = trait_def.help.strip()
        if help:
            schema[cls.S.DESC] = help
        match trait_def:
            case T.Unicode():
                schema.update(type="string")
                if hasattr(trait_def, "pattern"):
                    schema.update(pattern=trait_def.pattern)
            case T.Bool():
                schema.update(type="boolean")
            case TypedTuple():
                schema.update(
                    type="array", items=cls.trait_to_schema(trait_def._trait, defs)
                )
            case T.Tuple():
                schema.update(
                    type="array",
                    items=[
                        cls.trait_to_schema(value_trait_def, defs)
                        for value_trait_def in trait_def._traits
                    ],
                )
            case T.Dict():
                schema.update(type="object")
                if trait_def._value_trait:
                    schema.update(
                        additionalProperties=cls.trait_to_schema(
                            trait_def._value_trait, defs
                        )
                    )
            case InstanceDict():
                cls.dict_like_to_schema(schema, trait_def, defs)
            case T.Instance():
                cls.dict_like_to_schema(schema, trait_def, defs)
            case T.Union():
                schema.update(
                    anyOf=[
                        cls.trait_to_schema(t_t, defs) for t_t in trait_def.trait_types
                    ]
                )
            case unhandled:
                pprint.pprint({trait_def: trait_def.__dict__})
        return schema

    def to_dict(self):
        traits = self.traits()
        as_dict = {}

        for key, value in self.trait_values().items():
            if value is None or hasattr(value, "__len__") and not len(value):
                continue
            if traits[key].metadata.get("schema") == False:
                continue
            if hasattr(value, "to_dict"):
                as_dict[key] = value.to_dict()
                continue
            as_dict[key] = value
        return as_dict

In [ ]:
def show_schema(cls: "Type[HasSchema]", instance: "HasSchema"):
    from IPython.display import Markdown

    schema_dict = cls.to_schema(instance=instance)
    display(Markdown("\n".join(["```yaml", yaml.safe_dump(schema_dict), "```"])))

In [ ]:
def write_schema(dest: Path, cls: "Type[HasSchema]", instance=None):
    schema = cls.to_schema(instance=instance)
    schema_json = json.dumps(schema, indent=2, sort_keys=True)
    dest.parent.mkdir(exist_ok=True, parents=True)
    dest.write_text(schema_json, encoding="utf-8")

Eject to the `pip` ecosystem.

In [ ]:
class PipDeps(HasSchema):
    pip = TypedTuple(
        T.Unicode(),
        help="a PEP 508 description of a pip dependency",
    )

A description of the conda environment for doing work.

In [ ]:
class CondaEnv(HasSchema):
    parent = T.Instance("jupyak.tasks._yak.Yak").tag(schema=False)
    variables = T.Dict(
        T.Unicode(),
        T.Unicode(),
        help="environment variables to set for all repos when the work environment is created",
    )
    python_version = UnicodeWithRegex(
        "3.11",
        pattern=r"^3\.(8|9|11|10|11|12)$",
        help="the major version of python 3 to install",
    )
    nodejs_version = UnicodeWithRegex(
        "20", pattern="^(20)$", help="the major version of nodejs to install"
    )
    yarn_version = UnicodeWithRegex(
        "3.6",
        pattern=r"^3\.\d+",
        help="the major version of yarn to install",
    )
    conda_channels = TypedTuple(
        T.Unicode(),
        ("conda-forge", "nodefaults"),
        help="the priority-ordered set of conda URLs of channels to get packages",
    )
    conda_dependencies = TypedTuple(
        T.Unicode(),
        (
            "pip",
            "hatch",
            "hatch-jupyter-builder",
            "jupyter-packaging",
            "python-build",
            "python-libarchive-c",
            "jsonschema-with-format-nongpl",
        ),
        help="the conda-forge-compatible specs for additional packages to install",
    )
    pypi_deps = TypedTuple(
        T.Unicode(),
        help="the pypi-compatible specs for additional packages to install",
    )
    pypi_to_conda = T.Dict(
        help="names of PyPI distributions to replace with their conda-forge counterparts"
    )

    @T.default("pypi_to_conda")
    def _default_pypi_to_conda(self):
        return {
            "prometheus-client": "prometheus_client",
            "fastjsonschema": "python-fastjsonschema",
            "jupyterlab-pygments": "jupyterlab_pygments",
            "stack-data": "stack_data",
            "build": "python-build",
        }

    @property
    def environment_yml(self):
        return self.parent.work_path / W.ENVIRONMENT_YML

    @property
    def environment_yml_data(self):
        deps = [
            f"python =={self.python_version}.*",
            f"nodejs =={self.nodejs_version}.*",
            f"yarn =={self.yarn_version}.*",
            *self.conda_dependencies,
        ]
        if self.pypi_deps:
            deps["pip"] = self.pypi_deps
        env_yml = {
            "channels": self.conda_channels,
            "dependencies": deps,
            "variables": self.variables,
        }
        return env_yml

    @property
    def venv(self) -> Path:
        return self.parent.work_path / ".venv"

    @property
    def venv_history(self) -> Path:
        return self.venv / W.CONDA_META_HISTORY

    @property
    def run_args(self) -> list[str]:
        return ["conda", "run", "--prefix", str(self.venv), "--live-stream"]

    @property
    def bin(self) -> Path:
        return self.venv / "bin"

    @property
    def lib(self) -> Path:
        return self.venv / "lib"

    @property
    def share(self) -> Path:
        return self.venv / "share"

    @property
    def python3(self) -> list[Path]:
        return [*self.run_args, self.bin / W.PYTHON3]

    @property
    def py_site_packages(self) -> Path:
        return self.lib / f"python{self.python_version}/site-packages"

    @property
    def python_build(self) -> list[Path]:
        return [*self.python3, "-m", "build", "--no-isolation"]

    @property
    def pip(self) -> list[Path]:
        return [*self.python3, "-m", "pip"]

    @property
    def pip_editable(self) -> list[Path]:
        return [
            *self.pip,
            "install",
            "-vv",
            "--no-deps",
            "--ignore-installed",
            "--no-build-isolation",
            "--editable",
        ]

    @property
    def lab_share(self):
        return self.share / "jupyter/lab"

Information for how to clone the baseline and PRs for this repo

In [ ]:
class GitHub(HasSchema):
    _re_github = r"https://github\.com/.*"
    _re_mergeable = r"^(tree/[^s]+|pull/\d+|releases/tag/[^s]+)$"

    parent = T.Instance("jupyak.tasks._yak.Repo").tag(schema=False)
    url = UnicodeWithRegex(
        pattern=_re_github,
        help="the URL of the repo",
        allow_none=False,
    )
    baseline = UnicodeWithRegex(
        pattern=_re_mergeable,
        help="the URL of the baseline HEAD",
        allow_none=False,
    )
    merge_with = TypedTuple(
        UnicodeWithRegex(
            pattern=_re_mergeable,
            allow_none=False,
        ),
        help="an optional, ordered list of branches to merge into the `baseline`",
    )
    merge_strategy = T.Unicode(
        help="a custom `--strategy` to use during `git merge`", allow_none=True
    )
    merge_options = TypedTuple(
        T.Unicode(), help="extra options for (and requiring) the ``merge_strategy``"
    )

    @T.default("baseline")
    def _default_baseline(self):
        return "tree/main"

custom file targets

In [ ]:
class CustomTarget(HasSchema):
    glob_neighbor = T.Tuple(
        T.Unicode(help="a file glob, relative to the repo root"),
        T.Unicode(help="a path relative to match: refer to the match stem with {stem}"),
        help="a file glob and a relative template for an output file",
    )

a subset of the doit task definition

In [ ]:
class Task(HasSchema):
    name = T.Unicode(help="the name of the task")
    actions = TypedTuple(TypedTuple(T.Unicode()), help="shell tokens to provide to ")
    file_dep = TypedTuple(
        T.Unicode(),
        help="relative globs for files that, when changed, trigger a need to run this task",
    )
    targets = TypedTuple(
        T.Union([T.Unicode(), InstanceDict(CustomTarget)]),
        help="relative globs for files that, when changed, trigger a need to run this task",
    )
    needs_pth = TypedTuple(
        T.Unicode(),
        help="key of a repo that needs to be installed as a python .pth file before running this task",
    )

    def to_dict(self):
        as_dict = super().to_dict()
        targets = as_dict.get("targets", {})
        if targets:
            as_dict["targets"] = [
                target.to_dict() if hasattr(target, "to_dict") else target
                for target in targets
            ]
        return as_dict

JavaScript-related provisioning, building, and linking.


In [ ]:
class JSOptions(HasSchema):
    parent = T.Instance("jupyak.tasks._yak.Repo").tag(schema=False)
    dependencies = TypedTuple(
        T.Unicode(),
        help="the names of other members of `repos` that need to built and linked into this repo in the JS environment",
    )
    link_exclude_patterns = TypedTuple(
        T.Unicode(),
        help="regular expressions for the npm `@org/pkg` names that should _not_ be linked into this repo",
    )
    dist_exclude_patterns = TypedTuple(
        T.Unicode(),
        help="regular expressions for the paths in this repo that should _not_ be built and linked in other repos",
    )
    tasks = T.Dict(
        key_trait=T.Unicode(),
        value_trait=TypedTuple(T.Union([T.Unicode(), InstanceDict(Task)])),
        help="task templates required to fulfill various js needs",
    )
    install_exclude_resolutions = TypedTuple(
        T.Unicode(),
        help="regular expressions for the npm `@org/pkg` names that should _not_ be installed, ever, in this repo",
    )

    def to_dict(self):
        as_dict = super().to_dict()
        all_tasks = as_dict.get("tasks", {})
        if all_tasks:
            as_dict["tasks"] = {
                cwd: [task.to_dict() for task in path_tasks]
                for cwd, path_tasks in all_tasks.items()
            }
        return as_dict

    @property
    def package_jsons(self):
        return [self.root_package_json]

    @property
    def root_package_json(self):
        return self.parent.work_path / W.PACKAGE_JSON

    @property
    def yarn_state(self):
        return self.parent.work_path / W.YARN_STATE

    @property
    def yarn_lock(self):
        return self.parent.work_path / W.YARN_LOCK

    @property
    def all_install_exclude_resolutions(self):
        return sorted(
            set(
                [
                    *self.install_exclude_resolutions,
                    # test
                    "@stdlib/stats",
                    "canvas",
                    "playwright",
                    "jest",
                    "mocha",
                    "playwright-core",
                    "@playwright/test",
                    # build
                    "verdaccio",
                    # docs
                    "@microsoft/api-extractor",
                    "typedoc",
                    # linters
                    "yarn-deduplicate",
                    "lint-staged",
                    "@typescript-eslint/eslint-plugin",
                    "@typescript-eslint/parser",
                    "eslint",
                    "eslint-config-prettier",
                    "eslint-plugin-jest",
                    "eslint-plugin-prettier",
                    "eslint-plugin-react",
                    "prettier",
                    "stylelint",
                    "stylelint-config-prettier",
                    "stylelint-config-recommended",
                    "stylelint-config-standard",
                    "stylelint-csstree-validator",
                    "stylelint-prettier",
                ],
            ),
        )

Python-related provisioning, building, and linking.

In [ ]:
class PythonOptions(HasSchema):
    parent: "Repo" = T.Instance("jupyak.tasks._yak.Repo").tag(schema=False)
    modules = TypedTuple(
        T.Unicode(),
        help="the importable python names provided by this repo",
    )
    dependencies = TypedTuple(
        T.Unicode(),
        help="the names of other members of `repos` that need to built and linked into this repo in the Python environment",
    )
    file_dep = TypedTuple(
        T.Unicode(),
        help="files needed (usually created by js tasks) before and editable python install",
    )
    pyproject_tomls = TypedTuple(
        T.Unicode(),
        help="pyproject.toml files for installable packages",
    )
    setup_cfgs = TypedTuple(
        T.Unicode(),
        help="legacy setup.cfg files for installable packages",
    )
    lab_extensions = T.Dict(
        help="paths with extra file dependencies needed to build an extension in this repo",
    )

    @T.default("pyproject_tomls")
    def _default_pyproject_tomls(self):
        return [W.PYPROJECT_TOML]

    @property
    def labextension_script(self):
        return self.parent.parent.work_path / "scripts/labextension.py"

In [ ]:
class RepoLiteOptions(HasSchema):
    parent = T.Instance("jupyak.tasks._yak.Repo").tag(schema=False)
    wheel = T.Bool(False, help="whether to build and ship a noarch wheel for pyodide")
    wheel_file_dep = T.Dict(help="extra files needed to build a given wheel")
    needs_pth = TypedTuple(
        T.Unicode(),
        help="names of packages this repo provides that need to be installed before a lite site build",
    )
    skip_wheel_patterns = TypedTuple(T.Unicode(), help="paths to skip building wheels")

A description of a repo.

In [ ]:
class Repo(HasSchema):
    parent = T.Instance("jupyak.tasks._yak.Yak").tag(schema=False)
    name = T.Unicode(
        help="the duplicated name from the parent ``repos`` dictionary used for dependencies"
    )
    github = InstanceDict(GitHub)
    js = InstanceDict(JSOptions, allow_none=True)
    py = InstanceDict(PythonOptions, allow_none=True)
    lite = InstanceDict(RepoLiteOptions, allow_none=True)
    variables = T.Dict(
        T.Unicode(),
        T.Unicode(),
        help="repo-specific environment variables to set when building",
    )

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for child in [self.js, self.py, self.github, self.lite]:
            if child:
                child.parent = self

    @property
    def work_path(self):
        return self.parent.work_path / "repos" / self.name

    @property
    def run_context(self):
        work_path = self.work_path
        return work_path, {"cwd": work_path, "env": self.run_env}

    @property
    def run_env(self):
        work_path = self.parent.work_path
        env = dict(os.environ)
        env.update(self.variables)
        env.update(
            # use a single global yarn config...
            YARNRC=self.parent.yarnrc_path,
            # ... but per-project nx junk...
            NX_CACHE_DIRECTORY=work_path / ".cache/nx" / self.name,
            NX_PROJECT_GRAPH_CACHE_DIRECTORY=work_path / ".cache/nx-graph" / self.name,
            # avoid expensive pep 517 behavior
            SKIP_JUPYTER_BUILDER=1,
            HATCH_JUPYTER_BUILDER_SKIP_NPM=1,
            JUPYTER_PACKAGING_SKIP_NPM=1,
        )
        return {k: str(v) for k, v in env.items()}

options for the final jupyterlite build

In [ ]:
class LiteOptions(HasSchema):
    parent = T.Instance("jupyak.tasks._yak.Yak").tag(schema=False)

    gist = T.Unicode(
        help="a gist ID on GitHub to use as JupyterLite contents and config",
        allow_none=True,
    )

    @property
    def work_path(self) -> Path:
        return self.parent.work_path / "lite"

    @property
    def gist_path(self) -> Path:
        return self.parent.work_path / "repos" / "_lite_gist"

    @property
    def build_config_path(self) -> Path:
        return self.work_path / W.JUPYTER_LITE_CONFIG

    @property
    def run_config_path(self) -> Path:
        return self.work_path / W.JUPYTER_LITE_JSON

    @property
    def app_path(self) -> Path:
        return self.parent.dist_path / "lite"

    @property
    def app_shasums_path(self) -> Path:
        return self.app_path / W.SHA256SUMS

A description of a jupyak pull request

In [ ]:
class Yak(HasSchema):
    issue = T.Dict().tag(schema=False)

    conf_dir = T.Unicode().tag(schema=False)
    work_dir = T.Unicode().tag(schema=False)
    repos = T.Dict(
        value_trait=InstanceDict(Repo),
        help="git-hosted projects to clone, build, and use to deploy a JupyterLite site",
    )
    env = InstanceDict(
        CondaEnv, help="the conda environment to manage during the build process"
    )
    lite = InstanceDict(
        LiteOptions,
        help="JupyterLite-specific configuration for contents, building, and running",
    )

    _default_repos: dict[tuple[str, tuple[str]], TRepoFactory] = {}

    def __init__(self, issue: dict | None = None, **kwargs):
        issue = deepcopy(issue or Yak.find_config(conf_dir=kwargs.get("conf_dir")))
        kwargs["issue"] = deepcopy(
            nested_update({"repos": {}, "env": {}, "lite": {}}, issue)
        )
        kwargs["env"] = kwargs["issue"]["env"]
        kwargs["lite"] = kwargs["issue"]["lite"]
        super().__init__(**kwargs)
        for child in [self.env, self.lite]:
            child.parent = self
        for name, factory in self._sorted_default_repos():
            defaults = deepcopy(factory(self))
            issue_repo = deepcopy(self.issue["repos"].get(name, {}))
            self.repos[name] = Repo(**nested_update(defaults, issue_repo))
            self.repos[name].parent = self

    def to_dict(self):
        as_dict = super().to_dict()
        repos = as_dict.get("repos", {})
        if repos:
            as_dict["repos"] = {name: repo.to_dict() for name, repo in repos.items()}
        return as_dict

    @classmethod
    def finalize_schema(cls, schema: dict, defs: dict | None = None, instance=None):
        if not instance:
            return
        as_dict = instance.to_dict()

        props = schema[cls.S.PROPS]
        repos = props["repos"]
        addtl = repos[cls.S.ADD_PROPS]
        repo_ref = addtl.pop(cls.S.REF)
        one_of = [
            {cls.S.TITLE: "custom repo", "type": "object", cls.S.REF: repo_ref},
        ]
        one_of += [
            {
                cls.S.TITLE: name,
                cls.S.ALL_OF: [
                    {
                        "type": "object",
                        "properties": {"name": {"type": "string", "const": name}},
                    },
                    {cls.S.REF: repo_ref},
                ],
                cls.S.DEFAULT: repo.to_dict(),
            }
            for name, repo in sorted(instance.repos.items())
        ]
        addtl.update(
            {
                cls.S.ANY_OF: [
                    *one_of,
                ]
            }
        )

    @property
    def work_path(self) -> Path:
        return Path(self.work_dir)

    @property
    def cache_path(self) -> Path:
        return self.work_path / ".cache"

    @property
    def yarn_cache_path(self) -> Path:
        return self.cache_path / "yarn"

    @property
    def yarnrc_path(self) -> Path:
        return self.work_path / W.YARNRC

    @property
    def build_path(self) -> Path:
        return self.work_path / "build"

    @property
    def dist_path(self) -> Path:
        return self.work_path / "dist"

    @property
    def py_repos(self):
        return {name: repo for name, repo in self.repos.items() if repo.py}

    @property
    def js_repos(self):
        return {name: repo for name, repo in self.repos.items() if repo.js}

    @property
    def not_a_package_json(self):
        return self.work_path / "build/not-a-package/package.json"

    @T.default("work_dir")
    def _default_work_dir(self):
        work_dir = os.environ.get("JPYK_WORK_DIR")
        if work_dir is None:
            work_dir = Path(jupyak.__file__).parent.parent.parent / "work"
        return str(work_dir)

    @classmethod
    def _sorted_default_repos(cls):
        graph = {}
        repos = cls._default_repos
        named_factories = {}
        for (name, deps), factory in repos.items():
            named_factories[name] = factory
            graph[name] = set(deps)
        for name in graphlib.TopologicalSorter(graph).static_order():
            yield name, named_factories[name]

    @classmethod
    def repo(cls, name: str, needs: tuple[str] | None = None):
        key = (name, needs or ())

        def _ensure(default_factory: TRepoFactory):
            if key not in cls._default_repos:
                cls._default_repos[key] = default_factory

        return _ensure

    @classmethod
    def find_config(cls, conf_dir: str):
        issue_path = None
        conf_path = Path(conf_dir or Path.cwd())
        for candidate in [os.environ.get(W.ENV_VAR_CONFIG), *W.JPYK_CONFIGS]:
            if not candidate:
                continue
            issue_path = (conf_path / candidate).resolve()
            if issue_path.exists():
                break
        if issue_path and issue_path.exists():
            issue_text = issue_path.read_text(encoding="utf-8")
            suffix = issue_path.suffix
            if suffix in [".json"]:
                return json.loads(issue_text)
            if suffix in [".yml", ".yaml"]:
                return yaml.safe_load(issue_text)
            if suffix in [".toml"]:
                return tomllib.loads(issue_text)

            msg = f"{issue_path} exists, but could not be parsed"
            print(msg, file=sys.stderr)
            sys.exit(1)
        else:
            if not json.loads(
                os.environ.get(W.ENV_VAR_ALLOW_NO_CONFIG, "false").lower()
            ):
                msg = f"{W.ENV_VAR_CONFIG} resolved to missing file: {issue_path}"
                print(msg, file=sys.stderr)
                sys.exit(1)
            issue = {}
        return issue

and that's all